# Main example
#### Here will be described and realized the whole process of the algorithm. 

In [74]:
# preparations
import os
# change dir to the the root, where is the repo
os.chdir('/home/koritskiy/rqc/markowitz_partitioning')
import numpy as np
import time
from qboard.cimsim.cimsim import CIMSim
from qboard.cimsim import tuner
import qboard
import qboard.cache
from qboard.translators import markowitz
from qboard import qubo
from modules.partitioning import Partitioning
from modules.markowitz import Portfolio
from modules.recomposers import Recomposers

# make an output matrices look better
np.set_printoptions(precision=3,  # Digits after point
                    linewidth=170,  # Length of the line
                    suppress=True)  # Always fixed point notation (not scientific)
# fix randomness 
np.random.seed(4)

In this example we will be playing with covariance matrices that, after some permutation of rows and columns,
have block structure. In real life that rarely happens, but this way it would be easier to 
follow the logic of algorithm. 
Matrix that has submatrices of dimensions d1, d2, ... , dn on diagonal would be labled `block_dim = [d1,d2,...,dn]`. 
So let's create a block matrix and randomly premute it. 
* `block_mat` - block matrix before permutation;
* `mixed_mat` - block matrix after permutation;
* `mix_permutation_mat` - permutation matrix that was applied to `block_mat` to get `mixed_mat`.

In [75]:
block_dim = [3, 3, 4]
block_dim_name = "".join(str(n) for n in block_dim) # for paths
g_dim = sum(block_dim)
averages = np.random.rand(g_dim)
prices = np.random.rand(g_dim)

block_mat = Partitioning.rand_sym_block_gen(block_dim)
_, mixed_mat,_ = Partitioning.mixed_matrix_generator(block_mat=block_mat)
covariance = mixed_mat

# set parameters for markowitz problem in the following order : [averages, covariance, price]
theta = [0.5, 0.5, 0]

Given random block matrix:
[[ 0.619 -0.088  0.425  0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.244 -1.157  0.351  0.     0.     0.     0.     0.     0.     0.   ]
 [-0.182  1.898  0.723  0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.046 -0.983  0.054  0.     0.     0.     0.   ]
 [ 0.     0.     0.    -0.823 -1.209  2.223  0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.449  3.916 -1.113  0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     1.636 -1.361 -0.651  0.542]
 [ 0.     0.     0.     0.     0.     0.    -1.313 -2.358 -1.106  0.838]
 [ 0.     0.     0.     0.     0.     0.     1.437 -0.191 -0.276  0.797]
 [ 0.     0.     0.     0.     0.     0.    -0.601  1.348 -0.551 -0.009]]


Random permutation matrix:
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0

Now we create an object of `Portfolio` class that will build BQM model based on all the input for solving
markowitz portfolio problem and `Partitioning` that would find out how to break down a big covariance matrix into
the smaller ones. Do not confuse `theta` in these classes: in `Partitioning` it refers to the coefficient 
with penalty term when matrix is not permutation one. Look [original paper](https://1qbit.com/whitepaper/quantum-inspired-hierarchical-risk-parity/) for details.  



In [76]:
portfolio = Portfolio(theta=theta,
                      averages=averages,
                      prices=prices,
                      covariance=covariance)
part = Partitioning(mixed_mat=portfolio.covariance,
                    theta = 100)



Now we want to solve partitioning problem using Kerberos solver. For this purpose we import `KerberosSampler`
from modified module `cim_kerberos`. The switcher between dwave and simcim subproblem solver 
is `simcim` flag. 


In [77]:
from modules.cim_kerberos import KerberosSampler
response = KerberosSampler().sample(bqm = part.bqm, # bqm we want to solve
                                    max_subproblem_size=60, # subproblem size that is sent to subsolver (simcim or dwave)
                                    num_reads=2, # number of iterations of each branch
                                    simcim=False, # if True, use simcim, else send to dwave
                                    tuner_timeout=50, # how long (in seconds) wait for tuner to find params for simcim
                                    simcim_attempt_num=10000 # number of attempts of simcim solver
                                    )
solution = np.array([int(i) for i in response.first[0].values()])

# turn solution vector into matrix and assign it as a property of our problem
part.permutation_mat = part.list_to_mat(solution)

Begin of 0th iteration
End of 0th iteration
Begin of 1th iteration
End of 1th iteration


Now we want to check, whether solution matrix is permutation one. If not - simple bruteforce will
try to find best fix, but it won't work if number of mistakes is more than 8. 

(*It's possible to make this "fixer" better, but we assume that our mistake is small and if not - 
choose another solver*)

In [78]:
if not part.permutation_check(part.permutation_mat):
    # try to finx it by bruteforce
    new_solution_permutation_mat = part.to_permutation(part.permutation_mat)
    if part.permutation_check(new_solution_permutation_mat):
        part.permutation_mat = new_solution_permutation_mat
part.ordered_mat = part.permute(part.permutation_mat, part.mixed_mat)

# have a look at ordered matrix
part.ordered_mat

array([[ 0.046, -0.983,  0.054,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.823, -1.209,  2.223,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.449,  3.916, -1.113,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   , -0.009, -0.551,  1.348, -0.601,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.797, -0.276, -0.191,  1.437,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.838, -1.106, -2.358, -1.313,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.542, -0.651, -1.361,  1.636,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -1.157,  0.351,  0.244],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.898,  0.723, -0.182],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.088,  0.425,  0.619]])

Now we have a permutation, that makes our covariance matrix looks like block-diagonal.
We want to take these blocks and solve it separately, find solutions (here exactly) and concatenate it. 

In [79]:
# Get a list of smaller portfolios which has covariance, prices and averages according to permutation 
portfolios = Recomposers.permutation_decomposer(portfolio=portfolio, # main portfolio problem
                                                permutation_mat=part.permutation_mat,
                                                max_dim=(max(block_dim) + 1),
                                                theta=[0.5,0.5,0]) #maximal size of block to cut

# Solve each small portfolio with dwave_solver
solutions = []
for small_portfolio in portfolios:
    _, small_solution = small_portfolio.exact_solver()
    print(small_solution)
    solutions.append(small_solution)

# Apply permutation to concatenated solution to get the solution of the original portfolio
composed_solution = Recomposers.permutation_solution_composer(solutions, part.permutation_mat)
print("Solution with partitioning:")
portfolio.solution_energy(composed_solution), composed_solution


[1 1 0]
[0 0 1 1]
[1 0 0]
Solution with partitioning:


(-5.109080066994888, array([1, 1, 0, 1, 0, 0, 0, 1, 1, 0]))

In [80]:
# Now compare it with exact solution of the whole matrix
exact_solution = portfolio.exact_solver()
exact_solution

(-7.492128848664729, array([1, 1, 0, 1, 0, 0, 1, 1, 1, 0]))

In [81]:
# relative hamming distance between these solutions:
sum(i ^ j for i,j in zip(composed_solution, exact_solution[1]))/len(composed_solution)



0.1